## Neural Style Transfer

Source of Learning: [TensorFlow Tutorials](https://www.tensorflow.org/tutorials/generative/style_transfer)

### Importing Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import os
from PIL import Image
from skimage.io import imread, imshow, imread_collection, concatenate_images
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Add, Input, Flatten,Embedding,LSTM,GlobalAveragePooling2D,Concatenate,Conv2D,BatchNormalization,LeakyReLU,ZeroPadding2D,UpSampling2D,MaxPooling2D

### Reading and Visualising Images

In [ ]:
Image2 = pimg.imread('Images/Style.jpg')
Image1 = pimg.imread('Images/Me.jpg')

plt.imshow(Image1)
plt.show()
plt.imshow(Image2)
plt.show()

In [ ]:
Content = Image1/255.0
Style = Image2/255.0

### Downloading VGG Net

In [ ]:
tf.keras.backend.clear_session()
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

In [ ]:
vgg.summary()

### Loss Functions and Gram Matrices

Gram Matrices are computed for Style parts

$G_{c d}^{l}=\frac{\sum_{i j} F_{i j c}^{l}(x) F_{i j d}^{l}(x)}{I J}$

In [ ]:
def GramMatrix(style_output):
    Numerator = tf.linalg.einsum('bijc,bijd->bcd', style_output, style_output)
    input_shape = tf.shape(style_output)
    Denominator = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return Numerator/Denominator

Loss = $\alpha Loss_{Content} + \beta Loss_{Style}$

In [ ]:
def LossFunction(Content_Output,Style_Output,Content_Target,Style_Target,nclayers,nslayers,alpha=1e4,beta=1e-3):

    # Style Loss
    style_loss = tf.add_n([tf.reduce_mean((Style_Output[name]-Style_Target[name])**2) 
                           for name in Style_Output.keys()])
    style_loss *= beta / nclayers


    # Content Loss
    content_loss = tf.add_n([tf.reduce_mean((Content_Output[name]-Content_Targets[name])**2) 
                             for name in Content_Output.keys()])    
    content_loss *= alpha / nslayers


    loss = style_loss + content_loss
    return loss

### Building a Model

In [ ]:
Content_Layers = ['block5_conv2'] 

Style_Layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

nclayers = len(Content_Layers)
nslayers = len(Style_Layers)

In [ ]:
def StyleContentModel(Input,Content_Layers,Style_Layers):

    VGG_Data_Layers = [vgg.get_layer(name).output for name in Style_Layers+Content_Layers]
    Model = tf.keras.Model([vgg.input], VGG_Data_Layers)

    Input = tf.expand_dims(Input,axis=0) * 255.0
    Model_Outputs = Model(Input)

    Style_Outputs = {}
    Content_Outputs = {}

    i = 0
    for s in Style_Layers:
        Style_Outputs[s] = GramMatrix(Model_Outputs[i])
        i+=1

    for c in Content_Layers:
        Content_Outputs[c] = Model_Outputs[i]
        i+=1

    return Style_Outputs,Content_Outputs

In [ ]:
Style_Targets = StyleContentModel(Style,Content_Layers,Style_Layers)[0]
Content_Targets = StyleContentModel(Content,Content_Layers,Style_Layers)[1]

### Training the Model

In [ ]:
Optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

In [ ]:
def Clipping(ImageData):
    return tf.clip_by_value(ImageData, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
@tf.function()
def TrainingStep(ImageData):
    with tf.GradientTape() as tape:
        Style_Outputs,Content_Outputs = StyleContentModel(ImageData,Content_Layers,Style_Layers)
        Loss = LossFunction(Content_Outputs,Style_Outputs,Content_Targets,Style_Targets,nclayers,nslayers)

    gradients = tape.gradient(Loss, ImageData)
    Optimizer.apply_gradients([(gradients, ImageData)])
    ImageData.assign(Clipping(ImageData))

In [ ]:
ImageData = tf.Variable(Content)

In [ ]:
def Training(Epochs,Steps_per_Epoch):
    for i in range(Epochs):
        print ('Epochs:',i)
        for j in range(Steps_per_Epoch):
            TrainingStep(ImageData)

In [ ]:
Training(10,100)

### Image Generation

In [ ]:
ImageNData = ImageData.numpy()
plt.imshow(ImageNData)

In [ ]:
im = Image.fromarray(np.array(ImageNData*255,dtype=np.uint8))
im.save('Results/Styled.png')